# Wild Magic Surges
### A Comparison of Two Homebrew Methods

In order to make wild magic surges a more frequent occurence, we can tweak the rules to trigger them. Two such tweaks are:

1. The "Increasing Count" method. Start as usual with a Wild Magic Surge triggering when the player rolls a 1 on their Surge roll. Every time a Surge *does not occur*, increase the D.C. for avoiding the Surge by one: 1 $\rightarrow$ 2 $\rightarrow$ 3, etc. When a Surge *does* occur, reset the D.C. to 1.

2. The "Decreasing Dice" method. Start as usual with a Wild Magic Surge triggering when the player rolls a 1 on their d20 Surge roll. Every time a surge *does not occur*, decrease the size of the dice by one: d20 $\rightarrow$ d12 $\rightarrow$ d10, etc. When a Surge *does* occur, reset the dice to a d20. Here we stop at a d4 and just make the player continue rolling a d4 until they do get a Surge, but you could continue down to a coin flip, and you could even continue from a coin flip to an automatic Surge (rolling a 1 on a "d1").

Below we calculate the probabilities of triggering a Wild Magic Surge under both of the above systems.

In [165]:
import numpy as np
import pandas as pd
import altair as alt
dice = [20, 12, 10, 8, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
count = [*range(1,21)]
count

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

First, we build a list of probabilities for *not* rolling a 1 on each of the dice: d20, d12, d10, ..., d4

In [166]:
diceP = list(map(lambda x: 1-(1/x), dice))

We also build a list of probabilities for *not* rolling (1), (1/2), (1/2/3), etc. on a d20:

In [167]:
countP = list(map(lambda x: 1-(x/20), count))

Then we multiply the first $i$ probabilities together &mdash; for $i = 1,2,3, \dots , 20$ &mdash; to get the probability of going $i$ rolls without a Wild Magic Surge under either system:

In [168]:
diceCDF = []
countCDF = []
for i in range(1,21):
    diceCDF.append(np.prod(diceP[:i]))
    countCDF.append(np.prod(countP[:i]))

Lastly, because it's more intuitive to think about it this way, we subtract each of these probabilities from $1$ to get the probability of encountering a surge in $x$ number of rolls:

In [169]:
diceCDF = list(map(lambda x: 1-x, diceCDF))
countCDF = list(map(lambda x: 1-x, countCDF))

data = []
for i in range(0, len(diceCDF)):
    data.append([i+1, diceCDF[i], 'Decreasing Dice'])
    data.append([i+1, countCDF[i], 'Increasing Count'])

And now, the fun part, we plot the results!

In [191]:
#collapse
df = pd.DataFrame(data, columns=['Number of Rolls', 'Probability', 'Method'])
df.reset_index()

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Number of Rolls'], empty='none')

points = alt.Chart(df).mark_circle().encode(
    x='Number of Rolls:O',
    y=alt.Y('Probability', title='Probability of a Surge'),
    color='Method',
    opacity=alt.condition(nearest, alt.value(1), alt.value(.6))
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(df).mark_point().encode(
    x='Number of Rolls:O',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw text labels near the points, and highlight based on selection
text = points.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Probability:Q',alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(df).mark_rule(color='gray').encode(
    x='Number of Rolls:O',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    selectors, points, rules, text
).properties(
    width=600, height=300
)

alt.LayerChart(...)

Lastly, here is the same data, but in a table view:

In [164]:
df.pivot(index='Number of Rolls', columns='Method')

Probability                 
Method          Decreasing Dice Increasing Count
Number of Rolls                                 
1                      0.050000         0.050000
2                      0.129167         0.145000
3                      0.216250         0.273250
4                      0.314219         0.418600
5                      0.428516         0.563950
6                      0.571387         0.694765
7                      0.678540         0.801597
8                      0.758905         0.880958
9                      0.819179         0.934527
10                     0.864384         0.967264
11                     0.898288         0.985269
12                     0.923716         0.994107
13                     0.942787         0.997938
14                     0.957090         0.999381
15                     0.967818         0.999845
16                     0.975863         0.999969
17                     0.981897         0.999995
18                     0.986423         1.000000
19                     0.989817         1.000000
20                     0.992363         1.000000